# Spark Structured Streaming 

`Structured Streaming` — это масштабируемая и отказоустойчивая библиотека для потоковой обработки, построенный на базе `Spark SQL`. Основная идея - с потоковыми вычислениями можно работать так же, как и со статическими данными. 

In [1]:
import os
import random
import time
from typing import Iterator, List, Tuple

import dbldatagen as dg
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import window
from pyspark.sql.types import StringType

 # Если переменная окружения  `JAVA_HOME` не установлена, то тут можно её указать.
os.environ["JAVA_HOME"] = "/home/alex/java/jdk11"

Создаем сессию `Spark`, как обычно

In [3]:
spark = SparkSession \
    .builder \
    .appName("structured") \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

23/10/22 17:33:20 WARN Utils: Your hostname, burg resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/10/22 17:33:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/22 17:33:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Проверяем, создадим "статический" DataFrame

In [4]:
df = spark.createDataFrame([("row1", 10), ("row2", 200)], ["column1", "columns2"])

Модель исполнения:
1. Входные данные поступают пачками (`mini-batch`) и добавляются к некоторому "бесконечному" DataFrame. Размер и частота появления `mini-batch` зависит от источника (генерируются "по триггеру").
2. Пользователем описываются некоторые операции по преобразованию "бесконечного DataFrame", как в "статическом" Spark.
3. В итоге получается "результирующий DataFrame", который является результатом работы и записывается во внешний источник (топик Kafka, консоль, файлы, etc)

Создаем Streaming DataFrame, описывая процесс получения данных из какого-нибудь источника. Поддерживается 4 встроенных источника:
- Kafka (`kafka`)
- Файлы 
- Сеть (`socket`)
- Генерация DataFrame вида `(timestamp TIMESTAMP, value LONG )`, для тестовых целей (`rate`)

In [7]:
# Для Kafka нужно указать топик

# df = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", "localhost:9092") \
#   .option("subscribePattern", "topic*") \
#   .option("startingOffsets", "earliest") \
#   .load()


# Будет создаваться 10 записей в секунду 
df = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", "10") \
    .load()

df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



Можно запустить процесс обработки, дать поработать 10 секунд и остановить. Данные накапливаются в течение некоторого времени по триггеру в так называемый `mini-batch` и обрабатываются. Затем обновления добавляются в "бесконечный" DataFrame. Режим вывода может быть:
- `update` - выводить только обновленные строки
- `complete` - DataFrame полностью
- `append` - новые строки

Не все эти режимы доступны, зависит от применяемых операций обработки DataFrame. Результат будет выводиться в консоль. 

In [8]:


query = df \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(10)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2023-10-22 17:51:28.436|0    |
|2023-10-22 17:51:28.836|4    |
|2023-10-22 17:51:29.236|8    |
|2023-10-22 17:51:29.636|12   |
|2023-10-22 17:51:30.036|16   |
|2023-10-22 17:51:28.536|1    |
|2023-10-22 17:51:28.936|5    |
|2023-10-22 17:51:29.336|9    |
|2023-10-22 17:51:29.736|13   |
|2023-10-22 17:51:30.136|17   |
|2023-10-22 17:51:28.636|2    |
|2023-10-22 17:51:29.036|6    |
|2023-10-22 17:51:29.436|10   |
|2023-10-22 17:51:29.836|14   |
|2023-10-22 17:51:30.236|18   |
|2023-10-22 17:51:28.736|3    |
|2023-10-22 17:51:29.136|7    |
|2023-10-22 17:51:29.536|11   |
|2023-10-22 17:51:29.936|15   |
|2023-10-22 17:51:30.336|19  

Библиотека [dbldatagen](https://github.com/databrickslabs/dbldatagen) позволяет, для тестовых целей, генерировать DataFrame с заданной схемой и случайным содержимом. Создадим DataFrame с одной колонкой, в которой может быть одно из пяти заданных слов. 

In [9]:
# описываем данные, которые будут генерироваться
ds = dg.DataGenerator(spark, name="Words", rows=20, partitions=1) \
      .withColumn("word", StringType(), values=["hello", "world", "ok", "no", "yes"], weights=[1, 1, 2, 2, 2])

# создаем Streaming DataFrame
df = ds.build(withStreaming=True, options={'rowsPerSecond': 3})

df.printSchema()

root
 |-- word: string (nullable = false)



Теперь можно описать преобразования (подсчет слов) и выводить текущую статистику в консоль

In [10]:

df = df.groupBy("word").count()

query = df \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()


time.sleep(30)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|5    |
|ok   |8    |
|no   |8    |
|world|4    |
|yes  |8    |
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|7    |
|ok   |13   |
|no   |12   |
|world|7    |
|yes  |12   |
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|9    |
|ok   |18   |
|no   |17   |
|world|9    |
|yes  |16   |
+-----+-----+



23/10/22 17:58:01 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 4, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
23/10/22 17:58:01 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 4, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.


Проэмулируем получение данных от трех IoT-устройств

In [14]:
ds = dg.DataGenerator(spark, name="IOT", rows=1000, partitions=1) \
      .withColumn("time", "timestamp", expr="now()") \
      .withColumn("sensor", StringType(), values=["sensor_1", "sensor_2", "sensor_3"]) \
      .withColumn("value", "integer", minValue=0, maxValue=10, random=True)


df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

df.printSchema()

root
 |-- time: timestamp (nullable = false)
 |-- sensor: string (nullable = true)
 |-- value: integer (nullable = true)



In [15]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

df = df.groupBy("sensor").avg("value")

query = df \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(30)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+
|sensor|avg(value)|
+------+----------+
+------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+----------+
|sensor  |avg(value)|
+--------+----------+
|sensor_1|5.0       |
|sensor_2|5.85      |
|sensor_3|5.5       |
+--------+----------+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------+----------+
|sensor  |avg(value)|
+--------+----------+
|sensor_1|4.875     |
|sensor_2|5.55      |
|sensor_3|5.7       |
+--------+----------+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------+-----------------+
|sensor  |avg(value)       |
+--------+-----------------+
|sensor_1|4.701754385964913|
|sensor_2|5.228070175438597|
|sensor_3|5.642857142857143|
+--------+-----------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------+-----------------+
|sensor  |avg(value)       |
+--------+-----------------+
|sensor_1|4.972972972972973|
|sensor_2|5.36986301369863 |
|sensor_3|5.712328767123288|
+--------+-----------------+



23/10/22 18:01:29 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 5, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
23/10/22 18:01:29 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 5, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
23/10/22 18:01:29 ERROR ShuffleBlockFetcherIterator: Error occurred while fetching local blocks, null
23/10/22 18:01:29 ERROR Utils: Aborting task=====>              (145 + 2) / 200]
java.lang.IllegalStateException: Error committing version 6 into HDFSStateStore[id=(op=0,part=145),dir=file:/tmp/temporary-7c70df23-09d1-4199-87d4-05419f922538/state/0/145]
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider$HDFSBackedStateStore.commit(HDFSBackedStateStoreProvider.scala:148)
	at org.apache.spark.sql.execution.streaming.state.StreamingAggregationStateManagerBaseImpl.commit(StreamingAggregationStateManager.scala:90)
	at org.apache.spark.sql.exe

Можно считать статистику по "окнам", которые образуются заданными временными интервалами

In [17]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

windowed_df = df \
.groupBy(
    window(df.time, "10 seconds"),
    df.sensor
).avg("value")


query = windowed_df \
    .writeStream \
    .trigger(processingTime="5 seconds") \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(40)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+------+----------+
|window|sensor|avg(value)|
+------+------+----------+
+------+------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+--------+-----------------+
|window                                    |sensor  |avg(value)       |
+------------------------------------------+--------+-----------------+
|{2023-10-22 18:08:50, 2023-10-22 18:09:00}|sensor_1|4.882352941176471|
|{2023-10-22 18:08:50, 2023-10-22 18:09:00}|sensor_3|4.9375           |
|{2023-10-22 18:08:50, 2023-10-22 18:09:00}|sensor_2|5.470588235294118|
+------------------------------------------+--------+-----------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+--------+------------------+
|window                                    |sensor  |avg(value)        |
+------------------------------------------+--------+------------------+
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_1|4.882352941176471 |
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_3|4.142857142857143 |
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_3|4.9375            |
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_2|5.923076923076923 |
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_1|5.3076923076923075|
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_2|5.470588235294118 |
+------------------------------------------+--------+------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+--------+-----------------+
|window                                    |sensor  |avg(value)       |
+------------------------------------------+--------+-----------------+
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_1|4.882352941176471|
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_3|4.62962962962963 |
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_3|4.9375           |
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_2|5.076923076923077|
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_1|4.518518518518518|
|{2023-10-22 18:09:00, 2023-10-22 18:09:10}|sensor_2|5.470588235294118|
+------------------------------------------+--------+-----------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+--------+-----------------+
|window                                    |sensor  |avg(value)       |
+------------------------------------------+--------+-----------------+
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_3|4.615384615384615|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_1|4.882352941176471|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_3|4.62962962962963 |
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_3|4.9375           |
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_2|5.076923076923077|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_1|3.923076923076923|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_1|4.518518518518518|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_2|5.571428571428571|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_2|5.470588235294118|
+--------------------------------------

-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+--------+-----------------+
|window                                    |sensor  |avg(value)       |
+------------------------------------------+--------+-----------------+
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_3|4.333333333333333|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_1|4.882352941176471|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_3|4.62962962962963 |
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_3|4.9375           |
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_2|5.076923076923077|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_1|4.666666666666667|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_1|4.518518518518518|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_2|5.433333333333334|
|{2023-10-22 18:09:10, 2023-10-22 18:09:20}|sensor_2|5.470588235294118|
+--------------------------------------

-------------------------------------------
Batch: 6
-------------------------------------------
+------------------------------------------+--------+-----------------+
|window                                    |sensor  |avg(value)       |
+------------------------------------------+--------+-----------------+
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_3|4.333333333333333|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_1|4.882352941176471|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_3|4.62962962962963 |
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_3|4.9375           |
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_2|5.076923076923077|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_2|6.117647058823529|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_3|4.588235294117647|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_1|4.666666666666667|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_1|4.518518518518518|
|{2023-10-22 18:09:20, 2023-10-22 18:09

-------------------------------------------
Batch: 7
-------------------------------------------
+------------------------------------------+--------+-----------------+
|window                                    |sensor  |avg(value)       |
+------------------------------------------+--------+-----------------+
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_3|4.333333333333333|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_1|4.882352941176471|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_3|4.62962962962963 |
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_3|4.9375           |
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_2|5.076923076923077|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_2|5.147058823529412|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_3|4.515151515151516|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_1|4.666666666666667|
|{2023-10-22 18:09:20, 2023-10-22 18:09:30}|sensor_1|4.518518518518518|
|{2023-10-22 18:09:20, 2023-10-22 18:09

23/10/22 18:09:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 8, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
23/10/22 18:09:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 8, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
